In [ ]:
#Wenn man den typen bestimmte hat, sollte man zunächst alle Pflicht-Felder des Bibtex-Eintrages raussuchen und bestimmen.
#Dazu die Pflichtattribute in einer Schleife durchgehen.

In [1]:
import torch
import pandas as pd
torch.cuda.is_available()

True

In [25]:
from transformers import pipeline
ner_tagger = pipeline("ner", aggregation_strategy="simple")
text="G. H. Hinton, , Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948"
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)

df_PER = df_outputs[df_outputs["entity_group"] == "PER"]

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

In [26]:
df_PER

,entity_group,score,word,start,end
0,PER,0.999297,G,0,1
1,PER,0.920350,H. Hinton,3,12
2,PER,0.994751,Bengio,16,22
3,PER,0.998586,Y,24,25
4,PER,0.961921,LeCun,31,36
5,PER,0.998160,Y,38,39
8,PER,0.971820,J. Smith,95,103
9,PER,0.994571,J. Doe,106,112


In [24]:
startIndexPER = df_PER["start"].iloc[0]
endIndexPER = df_PER["end"].iloc[-1]

In [17]:
import pandas as pd

# Beispiel DataFrame erstellen
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Zugriff auf die erste Zeile
erste_zeile = df.iloc[0]
print(erste_zeile)

A    1
B    4
C    7
Name: 0, dtype: int64


In [42]:
#Test, um Datum Zuverlässig erkannt wird

import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948")
for ent in doc.ents:
  print(ent, ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_)

Hinton 0 6 GPE
G. 8 10 GPE
Bengio 12 18 PERSON
Y. 20 22 PERSON
Y. 34 36 PERSON
2021 38 42 DATE
Deep Learning for Artificial Intelligence 0 41 ORG
J. Smith & J. Doe (Eds 3 25 ORG
Proceedings of the IEEE International Conference on Neural Networks 29 96 ORG
1 103 104 CARDINAL
5 112 113 DATE
100–120 119 126 CARDINAL
IEEE Press 0 10 ORG
https://doi.org/10.1109/ICNN.2021.9483948 0 41 GPE


@book{advanced_physics, author = {Stephen Hawking and Brian Greene and Maria Trump and Susan Soy}, title = {Advanced Concepts in Theoretical Physics}, publisher = {Cambridge University Press}, year = {2025}, edition = {3}, volume = {1}, series = {Advanced Studies in Physics}, address = {Cambridge, UK}, month = {May}, note = {A seminal work in the field of theoretical physics}, isbn = {978-0-521-76948-0}, doi = {10.1017/example.book.2025}, url = {https://www.cambridge.org/advanced_physics}, annote = {Widely cited in the physics community}, abstract = {This book explores cutting-edge theories and concepts in theoretical physics...}, keywords = {Theoretical Physics, Quantum Mechanics, String Theory}, language = {English}, price = {75.00}, size = {600 pages}, lccn = {2020934576}, mrnumber = {MR3070071} }

@article{quantum_entanglement,
  author        = {Albert Einstein and Boris Podolsky and Nathan Rosen},
  title         = {Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?},
  journal       = {Physical Review},
  year          = {1935},
  volume        = {47},
  number        = {10},
  pages         = {777-780},
  month         = {May},
  note          = {EPR Paradox paper, fundamental for quantum mechanics},
  doi           = {10.1103/PhysRev.47.777},
  url           = {https://journals.aps.org/pr/abstract/10.1103/PhysRev.47.777},
  abstract      = {In this paper, the authors discuss the EPR paradox and challenge the completeness of quantum mechanics...},
  keywords      = {Quantum Mechanics, EPR Paradox, Physical Reality},
  language      = {English},
  publisher     = {American Physical Society}
}

@inproceedings{deep_learning,
  author        = {Geoffrey Hinton and Yoshua Bengio and Yann LeCun},
  title         = {Deep Learning for Artificial Intelligence},
  booktitle     = {Proceedings of the IEEE International Conference on Neural Networks},
  year          = {2021},
  editor        = {Jane Smith and John Doe},
  volume        = {1},
  number = {5},
  series        = {Advances in Neural Information Processing},
  pages         = {100-120},
  address       = {Montreal, Canada},
  month         = {December},
  organization  = {IEEE},
  publisher     = {IEEE Press},
  note          = {Keynote paper on recent advancements in deep learning},
  isbn          = {978-1-5386-4637-1},
  doi           = {10.1109/ICNN.2021.9483948},
  url           = {https://ieeexplore.ieee.org/document/9483948},
  annote        = {A seminal work on how deep learning transforms AI},
  abstract      = {This paper explores cutting-edge deep learning techniques and their impact on the development of artificial intelligence...},
  keywords      = {Deep Learning, Artificial Intelligence, Neural Networks},
  language      = {English}
}


@incollection{quantum_computation,
  author        = {Michael A. Nielsen and Isaac L. Chuang},
  title         = {Quantum Computation and Quantum Information},
  booktitle     = {Handbook of Quantum Information Science},
  publisher     = {Springer},
  year          = {2026},
  editor        = {Charles H. Bennett and David P. DiVincenzo},
  volume        = {4},
  series        = {Quantum Science and Technology},
  chapter       = {10},
  pages         = {250-300},
  address       = {Berlin, Germany},
  month         = {October},
  note          = {A comprehensive overview of the fundamentals of quantum computation},
  isbn          = {978-3-540-88702-7},
  doi           = {10.1007/springerreference_303198},
  url           = {https://www.springer.com/gp/book/9783540887027},
  annote        = {Essential reading for researchers entering the field of quantum information},
  abstract      = {This chapter delves into the principles of quantum computing, offering an accessible yet thorough introduction...},
  keywords      = {Quantum Computing, Quantum Information, Computational Models},
  language      = {English},
  price         = {45.00},
  size          = {50 pages}
}


In [3]:
#RegEx zum FInden der Seiten
pageFinder = ", (?:pp\.? )?\d+-\d+,"
doi1 = "https:\/\/doi\.org"
doi2 = "DOI:"

#if havard und book:
volume = ", \d+,"
number = ", \d+,"
edition1 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) ed\."
edition2 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) edn\."

#if APA or Havard then
volume = "\(Vol\. \d+" #für Volume und number noch als Bedingung, dass es nach Titel stehen muss, also Index > Titel-Index
number = "Issue \d+"
yearApaHavard = "\(\d{4}\)"
editorApaHavard = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors|edited by )(\))?"

#editor bei anderen


#Beachte: Editor und Edition werden oft mit gleichen Abkürzungen versehen, auch im gleichen Stil! Nur Edition wird oft
#klein geschrieben. Daher zustzlich prüfen, ob NER das als Person erkennt.

#Beachte: ieee und mla trennen mit , und nicht mit .
#if ieee and mla
volumeIeeeMla = ", vol\. \d+" 
noIeeeMla = ", no\. \d+"

etAl = "et al."

yearACM = "\. \d{4}\."

#Man sollte zunächst die Felder extrahieren, die zu 100% sicher erkennen kann wie Volume oder Number, doi, Autoren...-> 
#Titel, booktitle, series und journal danach

#man sollte eine Überdeckungsprüfung machen: Also Bspw. sagt Space im Bereich 25-50 ist eine Orga, Regex sagt, 
#dort ist ein (Eds.)
#zu finden und Huggingface sagt, von 25-45 ist was. Dann sollte der minimalteste Wert und maximalste Wert 
#genommen werden und so lange nach links und analog nach rechts gehen, bis wieder ein Punkt kommt.
# Da erkannte Felder aus dem String entfernt werden, muss der String am Ende leer sein. 
#Daher sollten zunächste die Dinge ausgeschnitten werden wo der Algo sich am sichersten ist
#TODO: Grundsätzliche Reihenfolge der BibTex-Einträge ermitteln.
#TODO: Für jede Extrahierung eines Feldes eine eigene Unterroutine schreiben? -> In der Oberschleife sollte Style sein





In [4]:
import re

txt = "Tpp. 250–300). Springer. https://doi.org/10.1007/springerreference_303198"
x = re.search(doi1, txt)
startIndex = x.start()
endIndex = x.end()

print("The first white-space character is located in position:", x.start()) 


The first white-space character is located in position: 25


In [10]:
listRegEx = [pageFinder, doi1]
for regex in listRegEx:
    x = re.search(regex, txt)
    if x:
        startIndex = x.start()
        endIndex = x.end()
        print(startIndex)

25


Test Parser APA + Book

In [47]:
import re

ner_tagger = pipeline("ner", aggregation_strategy="simple")
text="Hawking, S., Greene, B., Trump, M. and Soy, S. (2025). Advanced Concepts and John Miller in Theoretical Physics (3rd ed., Vol. 1). Cambridge University Press. https://doi.org/10.1017/example.book.2025"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chat GPT & Google Bard AI: A Review. 2023 International Conference on IoT, Communication and Automation Technology (ICICAT), 1–6. https://doi.org/10.1109/ICICAT57735.2023.10263706"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chapter Title. In E. F. Johnson & R. H. Lee (Eds.), Book Title. Publisher Name."
#text = "Singh, S. K. and Peterman, P. (2023). Chapter and Title. Trump, M. and Soy, S. Book Title. Publisher Name."
#text = "S. K. Singh, S. Kumar & P. S. Mehra, (2023). Chapter Title. In E. F. Johnson & R. H. Lee (Eds.), Book Title. Publisher Name."

outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)

#index neu setzen, da diese nicht automatich geupdates werden
df_PER = df_outputs[df_outputs["entity_group"] == "PER"].reset_index(drop=True)
df_PER_len = len(df_PER.index)


print(df_PER)

fullNameInOneWord = False
allFullNamesInOneWord = True
nameAbgekürzt = False
surenameFirst = False

for index in df_PER.index.values.tolist():
    if "." == text[df_PER["end"].iloc[index]] and len(text[df_PER["start"].iloc[index]:df_PER["end"].iloc[index] + 1]) == 2:
        nameAbgekürzt = True
for element in text:
    if element == ",":
        break
    if element in [".", " "]:
        surenameFirst = True


editorRegEx = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors)(\))?"
startIndexEditorMarker = -1
if re.search(editorRegEx, txt):
    startIndexEditorMarker = x.start()


startIndexPER = df_PER["start"].iloc[0]
endIndexPER = df_PER["end"].iloc[-1] + 1
#Beachte, ein ., auch zwischen Autoren und Titel vorkommen kann
authors = text[startIndexPER:endIndexPER]
editors = ""
andInAuthors = False
dotKommaInAuthors = False
firstStartIndexAnd = -1
firstEndIndexAnd = -1
search_terms = [" and ", ", and ", " & ", ", & "]

if "et al." not in text:
    firstStartIndexAnd, firstEndIndexAnd, andTyp = find_First_And_index(authors, search_terms)
    if firstStartIndexAnd > -1:
        andInAuthors = True
else:
    print("et. al")

if not surenameFirst:
    if authors.find(".,") > -1:
        dotKommaInAuthors = True


#prüfe, wo der letzte Autor in der Autorenkette ist, um Namen auszuschließen, die im Titel sein können
#vorher prüfen, wo et al. steht, weil so erkennt man ende sicher
#zusäzich kann man noch prüfen, wo das erst "and" oder "&" kommt


index_df_PER_List = df_PER.index.values.tolist()

distanceAuthorNames = 0
maxDistanceAuthorNames = 0
startDistanceAuthorNames = 0
endDistanceAuthorNames = 0
authorSeparatedByAnd = False
possibleEditors = False
andTyp = ""

firstStartIndexAnd = -1
firstEndIndexAnd = -1
listIndicesAnd = []
for index in index_df_PER_List[:-1]:
    textBetweenNames = text[df_PER["end"].iloc[index]:df_PER["start"].iloc[index + 1]]
    firstStartIndexAnd, firstEndIndexAnd, andTyp = find_First_And_index(textBetweenNames, search_terms)
    distanceAuthorNames = df_PER["start"].iloc[index + 1] - df_PER["end"].iloc[index]
    if firstStartIndexAnd > -1:
        listIndicesAnd.append([df_PER["end"].iloc[index], df_PER["start"].iloc[index + 1], andTyp])
    if distanceAuthorNames > maxDistanceAuthorNames:
        maxDistanceAuthorNames = distanceAuthorNames
        #die endDistanceAuthorNames braucht man, um dann den STart der Editors zu finden
        endIndexMaxDistanceAuthorNames = df_PER["start"].iloc[index + 1]
        startIndexMaxDistanceAuthorNames = df_PER["end"].iloc[index]

print("listIndicesAnd {0}".format(listIndicesAnd))
if len(listIndicesAnd) > 0:
    authorSeparatedByAnd = True
    for element in listIndicesAnd:
        if maxDistanceAuthorNames > (element[1] - element[0]):
            #Dann gibt es zwischen den Namen von Autoren eine größere Lücke als ein And. Irgendein And muss aber vorhanden sein.
            #endDistanceAuthorNames und startDistanceAuthorNames zeigen dann Range der Lücke an
            possibleEditors = True
else:
    # Dann kann der maximalste Abstand "., " sein
    if maxDistanceAuthorNames > 3:
        possibleEditors = True

if possibleEditors:
    authors = text[startIndexPER:startIndexMaxDistanceAuthorNames + 1]
    editors = text[endIndexMaxDistanceAuthorNames:endIndexPER]
else:
    authors = text[startIndexPER:endIndexPER]

andInAuthors = find_First_And_index(authors, search_terms)[2]
andInEditors = find_First_And_index(editors, search_terms)[2]
finalAuthors = ""
finalEditors = ""

if surenameFirst:
    #hier völlig egal, ob er einzelne Initialen in ein eigenes Word gesteckt hat, obwohl es noch Nachnamen gib
    andInAuhtors = find_First_And_index(authors, search_terms)[2]
    authors = authors.replace(andInAuthors, " and ")
    finalAuthors = authors.replace(", ", " and ")
elif andInAuthors != "":
    if "., " in authors:
        print("Fall ., {0}".format(authors))
        search_terms = ["., and ", "., & ", ". and ", ". & "]
        andInAuthors = find_First_And_index(authors, search_terms)[2]
        authors = authors.replace(andInAuthors, "., ")
        authors = authors.split("., ")
        authors = [name + "." for name in authors]
        authors = [name.replace("..",".") for name in authors]
        for author in authors[:-1]:
            buffer = author.split(", ")
            finalAuthors = finalAuthors + buffer[1] + " " + buffer[0] + " and "
        buffer = authors[-1].split(", ")
        finalAuthors = finalAuthors + buffer[1] + " " +  buffer[0]
    elif ", " in authors:
        search_terms = [", and ", ", & ", " and ", " & "]
        andInAuthors = find_First_And_index(authors, search_terms)[2]
        authors = authors.replace(andInAuthors, ", ")
        authors = authors.split(", ")
        for i in range(0, len(authors) - 3, 2):
            finalAuthors = finalAuthors + authors[i+1] + " " + authors[i] + " and "
        finalAuthors = finalAuthors + authors[len(authors) - 1] + " " + authors[len(authors) - 2]
else:
    print("Fall else {0}".format(authors))
    authors = authors.split(", ")
    finalAuthros = authors[1] + authors[0]
        

print(finalAuthors)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

   entity_group     score        word  start  end
0           PER  0.999392       Singh      0    5
1           PER  0.997331           S      7    8
2           PER  0.998019           K     10   11
3           PER  0.999534       Kumar     14   19
4           PER  0.998208           S     21   22
5           PER  0.984761       Mehra     27   32
6           PER  0.999057           P     34   35
7           PER  0.998489           S     37   38
8           PER  0.837229           E     66   67
9           PER  0.862536  F. Johnson     69   79
10          PER  0.992032           R     82   83
11          PER  0.965553      H. Lee     85   91
listIndicesAnd [[22, 27, ', & '], [79, 82, ' & ']]
Fall ., Singh, S. K., Kumar, S., & Mehra, P. S.
S. K. Singh and S. Kumar and P. S. Mehra


In [6]:
print(len("Hawking, S., Greene, B., Trump, M.,  Soy., S."))

45


In [22]:
def find_First_And_index(text, search_terms):
    # Initialisiere mit einem hohen Wert
    min_index = float('inf')
    end_index = 0
    andTyp = ""
    
    # Suche jeden Suchbegriff in dem Text und behalte den kleinsten Index
    for term in search_terms:
        index = text.find(term)
        if index != -1 and index < min_index:
            min_index = index
            end_index = min_index + len(term) - 1
            andTyp = term
    
    # Wenn min_index unverändert ist, wurde keiner der Begriffe gefunden
    return (min_index, end_index, andTyp) if min_index != float('inf') else (-1, -1, "")

    

In [29]:
print("" == "")

True
